In [1]:
import pandas as pd
import numpy as np
import tarfile
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time


In [2]:
data_gene_ex = pd.read_csv("data-TCGA.csv")

In [3]:
df_gene = pd.DataFrame(data_gene_ex)

In [10]:
data_label = pd.read_csv("labels-TCGA.csv")

In [11]:
df_label = pd.DataFrame(data_label)

In [16]:
df_label.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
df_label['Class'].replace({'BRCA':0, 'KIRC':1, "LUAD":2, "PRAD":3, "COAD":4}, inplace=True)

In [18]:
df_gene1 = df_gene.copy()
df_gene1 = df_gene1.drop(columns=['Unnamed: 0'])

In [19]:
df_gene2 = df_gene1.copy()
df_gene2 = df_gene2.T.drop_duplicates().T

In [20]:
x = df_gene2
y = df_label.values.reshape(-1,1)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [22]:
# feature scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_scaled = scaler.transform(x)

# Removing Low Variance Features

In [23]:
# Calculate variance for each gene
gene_variances_sorted = x_train.var(axis=0).sort_values(ascending=False)

In [24]:
# Keep top N genes
N = 2000
top_genes = gene_variances_sorted.index[:N]
x_train_top_var = x_train[top_genes]
x_test_top_var = x_test[top_genes]

x_new = x.copy()
x_new = x_new[top_genes]

print(f"Shape after keeping top {N} genes:", x_train_top_var.shape)
print(f"Shape after keeping top {N} genes:", x_test_top_var.shape)

Shape after keeping top 2000 genes: (640, 2000)
Shape after keeping top 2000 genes: (161, 2000)


# ANOVA

In [25]:
# Feature selection
selector = SelectKBest(f_classif, k=1000)
x_train_sel = selector.fit_transform(x_train_top_var, y_train)
x_test_sel = selector.transform(x_test_top_var)
x_sel = selector.transform(x_new)

C:\Users\AVIDEH\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
# Get the selected feature names
selected_feature_names = x_train_top_var.columns[selector.get_support()]

# Create DataFrames with selected features and their actual values
x_train_selected_df = pd.DataFrame(x_train_sel, columns=selected_feature_names)
x_test_selected_df = pd.DataFrame(x_test_sel, columns=selected_feature_names, index=x_test.index)
x_selected_df = pd.DataFrame(x_sel, columns=selected_feature_names, index=x.index)

x_test_selected_df

,gene_9176,gene_9175,gene_15898,gene_15301,gene_15589,gene_3540,gene_19661,gene_3541,gene_11250,gene_15897,...,gene_10106,gene_11772,gene_7998,gene_9564,gene_7474,gene_13152,gene_1059,gene_2572,gene_8906,gene_16450
697,0.000000,0.000000,19.557821,12.295416,0.000000,10.315229,1.966947,17.607518,8.492262,13.902987,...,0.000000,8.648354,14.415082,10.780622,0.000000,6.769018,0.000000,3.919464,5.759787,5.397361
668,0.404576,2.380868,0.404576,0.000000,15.367524,3.132495,10.971436,4.696395,1.556993,2.288299,...,0.000000,8.902378,8.544149,8.955086,5.461267,3.546351,0.000000,3.078234,7.512693,3.585780
63,1.078815,0.000000,19.807117,0.000000,0.000000,7.118225,13.151083,15.131552,5.958358,12.240374,...,1.078815,4.903086,14.716557,10.019188,0.000000,4.468231,0.000000,7.022723,7.492558,3.599734
534,0.000000,0.000000,1.896194,11.665056,2.345822,1.708673,9.608575,0.931607,1.708673,1.493084,...,4.672476,5.590569,6.694058,10.503826,1.896194,8.072165,5.053728,5.272725,10.921424,3.869042
66,0.000000,0.000000,0.744333,0.000000,3.385859,6.346090,14.759191,8.046229,0.744333,6.246967,...,0.000000,7.367004,6.618115,13.558074,7.441749,7.312892,0.744333,2.823138,6.390542,3.476161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,18.166516,16.418269,2.577175,13.105047,0.000000,6.620104,3.687374,6.120070,14.213750,6.810919,...,4.313050,4.035615,8.630922,12.041012,4.545641,4.745888,0.000000,7.745587,6.962618,4.990946
745,0.924100,0.000000,1.697640,0.000000,0.000000,15.754655,12.142551,13.938348,0.000000,3.032048,...,0.000000,9.981068,6.337611,3.556993,0.534759,0.924100,0.000000,0.000000,10.430641,2.864394
513,0.000000,4.221452,12.067323,11.769387,0.000000,15.385276,2.868924,15.945064,7.742242,8.588658,...,1.176961,8.071301,10.746430,10.322998,1.176961,6.996999,0.000000,7.754928,7.989480,5.255520
671,2.396954,1.351402,1.113500,0.000000,13.158630,5.461286,12.752947,5.716728,0.000000,1.555571,...,0.472904,7.452579,7.430043,10.538975,4.668885,6.489101,0.000000,5.594653,6.678565,3.310573


# LDA as dimension reduction

In [28]:
lda = LinearDiscriminantAnalysis(n_components=min(len(np.unique(y_train)) - 1, 10))
x_train_final = lda.fit_transform(x_train_selected_df, y_train)
x_test_final = lda.transform(x_test_selected_df)


C:\Users\AVIDEH\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
# svm model with different kernels and degree = 3 and gamma= 0.7


def svm(x_tr, x_te):
    
    for k in ['linear', 'rbf', 'poly']:
        
        svm_model = SVC(kernel=k, degree=3, gamma=0.7)
        start = time.time()
        svm_model.fit(x_tr, y_train.ravel())
        train_time = time.time() - start

        y_svm_pred = svm_model.predict(x_te)
        acc = accuracy_score(y_test, y_svm_pred)

        
        print(f"{k}, accuracy = {acc:.4f}, training_time = {train_time:.4f} sec")

In [33]:
svm(x_train_final,x_test_final)

linear, accuracy = 1.0000, training_time = 0.4000 sec
rbf, accuracy = 0.4099, training_time = 0.0650 sec
poly, accuracy = 0.9876, training_time = 0.0000 sec


In [38]:
svm_model1 = SVC(kernel='poly', degree=3, gamma=0.7)

svm_model1.fit(x_train_final, y_train.ravel())

y_svm_pred1 = svm_model1.predict(x_test_final)

In [39]:
# Report of svm with kernel = ploy and degree=3 and gamma=0.7
print(classification_report(y_test, svm_model1.predict(x_test_final)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        61
           1       1.00      1.00      1.00        25
           2       1.00      0.97      0.98        29
           3       1.00      0.97      0.98        29
           4       1.00      1.00      1.00        17

    accuracy                           0.99       161
   macro avg       0.99      0.99      0.99       161
weighted avg       0.99      0.99      0.99       161



In [40]:
# Report of svm with kernel = linear and degree=3 and gamma=0.7

svm_model2 = SVC(kernel='poly', degree=3, gamma=0.7)

svm_model2.fit(x_train_final, y_train.ravel())

y_svm_pred2 = svm_model2.predict(x_test_final)


print(classification_report(y_test, svm_model2.predict(x_test_final)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        61
           1       1.00      1.00      1.00        25
           2       1.00      0.97      0.98        29
           3       1.00      0.97      0.98        29
           4       1.00      1.00      1.00        17

    accuracy                           0.99       161
   macro avg       0.99      0.99      0.99       161
weighted avg       0.99      0.99      0.99       161



In [41]:
# Report of svm with kernel = linear and degree=3 and gamma=0.7

svm_model3 = SVC(kernel='rbf', degree=3, gamma=0.7)

svm_model3.fit(x_train_final, y_train.ravel())

y_svm_pred3 = svm_model3.predict(x_test_final)


print(classification_report(y_test, svm_model3.predict(x_test_final)))

              precision    recall  f1-score   support

           0       1.00      0.31      0.47        61
           1       1.00      0.40      0.57        25
           2       1.00      0.17      0.29        29
           3       0.23      1.00      0.38        29
           4       1.00      0.18      0.30        17

    accuracy                           0.41       161
   macro avg       0.85      0.41      0.40       161
weighted avg       0.86      0.41      0.42       161



In [43]:
def svm (x_tr, x_te):

      for k in ['linear', 'rbf', 'poly']:

        svm_model = SVC(kernel= k)

        start = time.time()

        svm_model.fit(x_tr, y_train.ravel())
        train_time = time.time() - start

        y_svm_pred = svm_model.predict(x_te)

        acc = accuracy_score(y_test, y_svm_pred)

        
        print(f"{k}, accuracy = {acc:.4f}, training_time = {train_time:.4f} sec")

        

svm(x_train_final,x_test_final)

linear, accuracy = 1.0000, training_time = 0.0051 sec
rbf, accuracy = 1.0000, training_time = 0.0000 sec
poly, accuracy = 0.9814, training_time = 0.0000 sec


In [44]:
# Report of svm with kernel = linear 

svm_model4 = SVC(kernel='linear')

svm_model4.fit(x_train_final, y_train.ravel())

y_svm_pred4 = svm_model4.predict(x_test_final)


print(classification_report(y_test, svm_model4.predict(x_test_final)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



In [45]:
# Report of svm with kernel = rbf

svm_model5 = SVC(kernel='rbf')

svm_model5.fit(x_train_final, y_train.ravel())

y_svm_pred5 = svm_model5.predict(x_test_final)


print(classification_report(y_test, svm_model5.predict(x_test_final)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        61
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        17

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



In [46]:
# Report of svm with kernel = poly 

svm_model6 = SVC(kernel='poly')

svm_model6.fit(x_train_final, y_train.ravel())

y_svm_pred6 = svm_model6.predict(x_test_final)


print(classification_report(y_test, svm_model6.predict(x_test_final)))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        61
           1       1.00      1.00      1.00        25
           2       1.00      0.93      0.96        29
           3       1.00      0.97      0.98        29
           4       1.00      1.00      1.00        17

    accuracy                           0.98       161
   macro avg       0.99      0.98      0.98       161
weighted avg       0.98      0.98      0.98       161



We can see that the models performance with LDA is poorer than without it. So, LDA in some models has negative effect.